In [1]:
import random
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import torchvision
import torchvision.transforms as transforms
import torch

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [2]:
def return_stats(guessed_labels, true_labels):    
    label_count = len(guessed_labels)
    negative_instance_count = label_count - sum(true_labels)
    
    threshold = np.sort(guessed_labels[:,0])[negative_instance_count]
    
    rounded_guessed_labels = np.zeros(label_count)
    
    print("there are:", sum(true_labels), "car images in the testing dataset, which has a total size of:", len(true_labels))
        
    for idx in range(label_count):
        if guessed_labels[idx] >= threshold:
            rounded_guessed_labels[idx] = 1
        else:
            rounded_guessed_labels[idx] = 0
            

    f1 = metrics.f1_score(rounded_guessed_labels, true_labels) 
    recall = metrics.recall_score(rounded_guessed_labels, true_labels)   
    precision = metrics.precision_score(rounded_guessed_labels, true_labels)  
    accuracy = metrics.accuracy_score(rounded_guessed_labels, true_labels)
    print(sum(rounded_guessed_labels), "images were labelled as a car image")

    print('f1 score, precision, recall, mean precision-recall', 'accuracy')
    return(np.array([f1, precision, recall, 0.5*(precision+recall), accuracy]))


In [3]:
# Load Caltech 101 data

# Load and preprocess the images and labels
data_dir = 'data/caltech101/caltech-101/101_ObjectCategories'



caltech_images = []
caltech_labels = []
for folder_name in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder_name)
    if folder_name == 'car_side':
        label = 1
    else:
        label = 0
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        img = image.load_img(file_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        caltech_images.append(img_array)
        caltech_labels.append(label)

# Convert lists to arrays
caltech_images = np.array(caltech_images)
caltech_labels = np.array(caltech_labels)

# Split the data into training and validation sets
caltech_X_train, caltech_X_val, caltech_y_train, caltech_y_val = train_test_split(caltech_images, caltech_labels, test_size=0.2, random_state=42)





In [3]:
# Load VOC data

# Load and preprocess the images and labels

# Directory paths
data_dir = 'data/VOC2007'
images_dir = os.path.join(data_dir, 'JPEGImages')
annotations_dir = os.path.join(data_dir, 'Annotations')

# Function to parse XML annotation files and extract object names
def parse_annotation(xml_file):
    import xml.etree.ElementTree as ET
    tree = ET.parse(xml_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall('object'):
        obj_name = obj.find('name').text
        objects.append(obj_name)
    return objects




# Load and preprocess the images and labels
voc_images_list = []
voc_labels_list = []

for filename in os.listdir(images_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(images_dir, filename)
        annotation_path = os.path.join(annotations_dir, filename[:-4] + '.xml')  # Replace extension with .xml

        if os.path.exists(annotation_path):
            objects = parse_annotation(annotation_path)
            if 'car' in objects:  # Check if 'car' object is present
                label = 1
            else:
                label = 0

            try:
                img = image.load_img(image_path, target_size=(224, 224))
                img_array = image.img_to_array(img)
                voc_images_list.append(img_array)
                voc_labels_list.append(label)
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")

# Convert lists to arrays
voc_images = np.array(voc_images_list)
voc_labels = np.array(voc_labels_list)

# Split the data into training and validation sets
voc_X_train, voc_X_val, voc_y_train, voc_y_val = train_test_split(voc_images, voc_labels, test_size=0.2, random_state=42)


In [4]:
# Load msrc data

# Load and preprocess the images and labels
data_dir = 'data/msrcorid'

msrc_images_list = []
msrc_labels_list = []


processed_files = set()

def process_folder(folder_path, label):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isdir(file_path):
            process_folder(file_path, label)  # Recursively process subdirectories
        else:
            # Check if file has already been processed
            if file_path not in processed_files:
                processed_files.add(file_path)  # Add file to processed set
                try:
                    img = image.load_img(file_path, target_size=(224, 224))
                    img_array = image.img_to_array(img)
                    msrc_images_list.append(img_array)
                    msrc_labels_list.append(label)
                except Exception as e:
                    print(f"Error loading image {file_path}: {e}")

# Loop through top-level directories
for folder_name in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder_name)
    if os.path.isdir(folder_path):
        if folder_name == 'cars':
            label = 1
        else:
            label = 0
        # Process the folder
        process_folder(folder_path, label)

# Convert lists to arrays
msrc_images = np.array(msrc_images_list)
msrc_labels = np.array(msrc_labels_list)

# Split the data into training and validation sets
msrc_X_train, msrc_X_val, msrc_y_train, msrc_y_val = train_test_split(msrc_images, msrc_labels, test_size=0.2, random_state=42)



In [ ]:
# Load models

In [5]:
caltech_model = load_model('models/caltech101_model_cnn_adj.keras')

In [6]:
voc_model = load_model('models/voc2007_model_cnn_adj.keras')

In [7]:
msrc_model = load_model('models/msrcorid_model_cnn_adj.keras')

In [ ]:
# Make predictions on test data:

In [7]:
tr_caltech_te_caltech_results = caltech_model.predict(caltech_X_val)
tr_voc_te_caltech_results = voc_model.predict(caltech_images)
tr_msrc_te_caltech_results = msrc_model.predict(caltech_images)


58/58 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step
286/286 ━━━━━━━━━━━━━━━━━━━━ 106s 325ms/step
286/286 ━━━━━━━━━━━━━━━━━━━━ 73s 248ms/step


In [7]:
tr_caltech_te_voc_results = caltech_model.predict(voc_images)
tr_voc_te_voc_results = voc_model.predict(voc_X_val)
tr_msrc_te_voc_results = msrc_model.predict(voc_images)


157/157 ━━━━━━━━━━━━━━━━━━━━ 79s 459ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 173ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 39s 222ms/step


In [8]:
tr_caltech_te_msrc_results = caltech_model.predict(msrc_images)
tr_voc_te_msrc_results = voc_model.predict(msrc_images)
tr_msrc_te_msrc_results = msrc_model.predict(msrc_X_val)


136/136 ━━━━━━━━━━━━━━━━━━━━ 38s 263ms/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step


In [ ]:
# Print results of predictions:

In [8]:
print(return_stats(tr_caltech_te_caltech_results, caltech_y_val))
print(return_stats(tr_voc_te_caltech_results, caltech_labels))
print(return_stats(tr_msrc_te_caltech_results, caltech_labels))


there are: 25 car images in the testing dataset, which has a total size of: 1829
129.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.25974026 0.8        0.15503876 0.47751938 0.93767086]
there are: 123 car images in the testing dataset, which has a total size of: 9145
881.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.10358566 0.42276423 0.05902384 0.24089403 0.90158557]
there are: 123 car images in the testing dataset, which has a total size of: 9145
123.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.08943089 0.08943089 0.08943089 0.08943089 0.97550574]


In [8]:
print(return_stats(tr_caltech_te_voc_results, voc_labels))
print(return_stats(tr_voc_te_voc_results, voc_y_val))
print(return_stats(tr_msrc_te_voc_results, voc_labels))



there are: 761 car images in the testing dataset, which has a total size of: 5011
5011.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.26368676 1.         0.1518659  0.57593295 0.1518659 ]
there are: 155 car images in the testing dataset, which has a total size of: 1003
155.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.36774194 0.36774194 0.36774194 0.36774194 0.80458624]
there are: 761 car images in the testing dataset, which has a total size of: 5011
761.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.24704336 0.24704336 0.24704336 0.24704336 0.77130313]


In [9]:
print(return_stats(tr_caltech_te_msrc_results, msrc_labels))
print(return_stats(tr_voc_te_msrc_results, msrc_labels))
print(return_stats(tr_msrc_te_msrc_results, msrc_y_val))



there are: 505 car images in the testing dataset, which has a total size of: 4323
659.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.         0.         0.         0.         0.73074254]
there are: 505 car images in the testing dataset, which has a total size of: 4323
595.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.44181818 0.48118812 0.40840336 0.44479574 0.857969  ]
there are: 122 car images in the testing dataset, which has a total size of: 865
122.0 images were labelled as a car image
f1 score, precision, recall, mean precision-recall accuracy
[0.91803279 0.91803279 0.91803279 0.91803279 0.97687861]
